In [1]:
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [2]:
print(len(documents))
print(documents[:5])

1186018
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


## Data Pre-processing
We will perform the following steps:
Tokenization: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
Words that have fewer than 3 characters are removed.
All stopwords are removed.
Words are lemmatized — words in third person are changed to first person and verbs in past and future tenses are changed into present.
Words are stemmed — words are reduced to their root form.

In [3]:
# Loading gensim and nltk libraries
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
import nltk.stem as stemmer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [5]:
## Write a function to perform lemmatize and stem preprocessing steps on the data set.
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [6]:
#Select a document to preview after preprocessing.
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


 tokenized and lemmatized document: 
['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']


In [7]:
#Preprocess the headline text, saving the results as ‘processed_docs’

In [8]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [9]:
#Bag of Words on the Data set
#Create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set.
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [10]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [11]:
#Gensim doc2bow
#For each document we create a dictionary reporting how many
#words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.

In [12]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(162, 1), (240, 1), (292, 1), (589, 1), (839, 1), (3575, 1), (3576, 1)]

In [13]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 162 ("govt") appears 1 time.
Word 240 ("group") appears 1 time.
Word 292 ("vote") appears 1 time.
Word 589 ("local") appears 1 time.
Word 839 ("want") appears 1 time.
Word 3575 ("compulsori") appears 1 time.
Word 3576 ("ratepay") appears 1 time.


In [14]:
#TF-IDF
#Create tf-idf model object using models.
#TfidfModel on ‘bow_corpus’ and save it to ‘tfidf’, then apply transformation to the entire corpus
#and call it ‘corpus_tfidf’. Finally we preview TF-IDF scores for our first document.

In [15]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5860586362613176),
 (1, 0.3854657616151764),
 (2, 0.5006618583937537),
 (3, 0.5072367544211179)]


In [16]:
#Running LDA using Bag of Words
#Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)


In [17]:
#For each topic, we will explore the words occuring in that topic and its relative weight.
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.026*"market" + 0.022*"school" + 0.015*"investig" + 0.015*"interview" + 0.014*"street" + 0.013*"fall" + 0.013*"student" + 0.011*"scott" + 0.011*"share" + 0.011*"bushfir"
Topic: 1 
Words: 0.020*"donald" + 0.018*"plan" + 0.018*"canberra" + 0.015*"water" + 0.013*"feder" + 0.013*"council" + 0.012*"speak" + 0.012*"price" + 0.011*"chang" + 0.010*"drum"
Topic: 2 
Words: 0.041*"australia" + 0.037*"trump" + 0.024*"world" + 0.016*"open" + 0.014*"tasmania" + 0.014*"final" + 0.013*"australian" + 0.012*"win" + 0.011*"break" + 0.010*"game"
Topic: 3 
Words: 0.029*"queensland" + 0.021*"help" + 0.017*"miss" + 0.016*"royal" + 0.015*"tasmanian" + 0.014*"bank" + 0.013*"victoria" + 0.013*"report" + 0.012*"commiss" + 0.011*"drought"
Topic: 4 
Words: 0.023*"news" + 0.022*"women" + 0.021*"live" + 0.020*"warn" + 0.020*"coast" + 0.019*"health" + 0.016*"rural" + 0.015*"countri" + 0.015*"busi" + 0.013*"gold"
Topic: 5 
Words: 0.054*"polic" + 0.050*"say" + 0.024*"kill" + 0.023*"attack" + 0.021*"cr

In [18]:
#Running LDA using TF-IDF
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.013*"coast" + 0.009*"gold" + 0.006*"outback" + 0.005*"coal" + 0.005*"australia" + 0.005*"queensland" + 0.005*"project" + 0.004*"destroy" + 0.004*"social" + 0.004*"budget"
Topic: 1 Word: 0.017*"crash" + 0.015*"drum" + 0.010*"tuesday" + 0.009*"christma" + 0.008*"sport" + 0.008*"juli" + 0.008*"road" + 0.008*"die" + 0.007*"financ" + 0.007*"truck"
Topic: 2 Word: 0.013*"search" + 0.012*"miss" + 0.007*"mount" + 0.006*"billion" + 0.006*"data" + 0.006*"wrap" + 0.006*"heatwav" + 0.006*"chines" + 0.006*"rate" + 0.006*"run"
Topic: 3 Word: 0.027*"trump" + 0.022*"news" + 0.018*"market" + 0.015*"rural" + 0.014*"donald" + 0.011*"royal" + 0.009*"commiss" + 0.008*"michael" + 0.007*"nation" + 0.007*"share"
Topic: 4 Word: 0.013*"elect" + 0.009*"govern" + 0.008*"labor" + 0.008*"turnbul" + 0.007*"say" + 0.007*"liber" + 0.007*"morrison" + 0.007*"feder" + 0.007*"sexual" + 0.006*"parti"
Topic: 5 Word: 0.009*"health" + 0.008*"friday" + 0.007*"thursday" + 0.006*"mental" + 0.006*"explain" + 0.005

In [19]:
processed_docs[4310]

['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']

In [20]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.6173104047775269	 
Topic: 0.020*"donald" + 0.018*"plan" + 0.018*"canberra" + 0.015*"water" + 0.013*"feder" + 0.013*"council" + 0.012*"speak" + 0.012*"price" + 0.011*"chang" + 0.010*"drum"

Score: 0.15054121613502502	 
Topic: 0.014*"show" + 0.013*"liber" + 0.012*"morrison" + 0.012*"wall" + 0.012*"young" + 0.011*"station" + 0.010*"christma" + 0.010*"beach" + 0.010*"research" + 0.009*"shark"

Score: 0.1445961594581604	 
Topic: 0.023*"news" + 0.022*"women" + 0.021*"live" + 0.020*"warn" + 0.020*"coast" + 0.019*"health" + 0.016*"rural" + 0.015*"countri" + 0.015*"busi" + 0.013*"gold"

Score: 0.01250994112342596	 
Topic: 0.020*"south" + 0.018*"adelaid" + 0.017*"north" + 0.015*"protest" + 0.014*"labor" + 0.011*"west" + 0.011*"worker" + 0.010*"state" + 0.009*"stori" + 0.009*"polit"

Score: 0.012507245875895023	 
Topic: 0.034*"year" + 0.031*"charg" + 0.028*"court" + 0.026*"murder" + 0.018*"face" + 0.017*"jail" + 0.016*"accus" + 0.015*"death" + 0.014*"peopl" + 0.013*"trial"

Score: 0.012

In [21]:
#Performance evaluation by classifying sample document using LDA TF-IDF model.
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))



Score: 0.526769757270813	 
Topic: 0.013*"elect" + 0.009*"govern" + 0.008*"labor" + 0.008*"turnbul" + 0.007*"say" + 0.007*"liber" + 0.007*"morrison" + 0.007*"feder" + 0.007*"sexual" + 0.006*"parti"

Score: 0.21694372594356537	 
Topic: 0.027*"trump" + 0.022*"news" + 0.018*"market" + 0.015*"rural" + 0.014*"donald" + 0.011*"royal" + 0.009*"commiss" + 0.008*"michael" + 0.007*"nation" + 0.007*"share"

Score: 0.1687326580286026	 
Topic: 0.013*"search" + 0.012*"miss" + 0.007*"mount" + 0.006*"billion" + 0.006*"data" + 0.006*"wrap" + 0.006*"heatwav" + 0.006*"chines" + 0.006*"rate" + 0.006*"run"

Score: 0.012510406784713268	 
Topic: 0.009*"health" + 0.008*"friday" + 0.007*"thursday" + 0.006*"mental" + 0.006*"explain" + 0.005*"govern" + 0.005*"august" + 0.005*"emerg" + 0.005*"marriag" + 0.005*"servic"

Score: 0.012508762069046497	 
Topic: 0.013*"coast" + 0.009*"gold" + 0.006*"outback" + 0.005*"coal" + 0.005*"australia" + 0.005*"queensland" + 0.005*"project" + 0.004*"destroy" + 0.004*"social" + 0.

In [22]:
#Testing model on unseen document
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.5166944265365601	 Topic: 0.036*"elect" + 0.033*"govern" + 0.026*"china" + 0.014*"australia" + 0.012*"million"
Score: 0.18335597217082977	 Topic: 0.020*"south" + 0.018*"adelaid" + 0.017*"north" + 0.015*"protest" + 0.014*"labor"
Score: 0.18324922025203705	 Topic: 0.054*"polic" + 0.050*"say" + 0.024*"kill" + 0.023*"attack" + 0.021*"crash"
Score: 0.016671964898705482	 Topic: 0.014*"show" + 0.013*"liber" + 0.012*"morrison" + 0.012*"wall" + 0.012*"young"
Score: 0.016671407967805862	 Topic: 0.026*"market" + 0.022*"school" + 0.015*"investig" + 0.015*"interview" + 0.014*"street"
Score: 0.016671407967805862	 Topic: 0.020*"donald" + 0.018*"plan" + 0.018*"canberra" + 0.015*"water" + 0.013*"feder"
Score: 0.016671407967805862	 Topic: 0.041*"australia" + 0.037*"trump" + 0.024*"world" + 0.016*"open" + 0.014*"tasmania"
Score: 0.016671407967805862	 Topic: 0.029*"queensland" + 0.021*"help" + 0.017*"miss" + 0.016*"royal" + 0.015*"tasmanian"
Score: 0.016671407967805862	 Topic: 0.023*"news" + 0.022